In [ ]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import pypsa
import yaml

### Scenario selection

In [ ]:
h2export = 200
EXPORT = h2export
CO2L = ""
CLUSTERS = 10
H = 3
LL = "1.0"
DISCOUNT_RATE = 0.071

### Settings

In [ ]:


#RUN = "decr_13_3h_ws"
CATEGORY = "postnetworks"
#SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_{opts}_{H}H_2030_{DISCOUNT_RATE}_DF_{EXPORT}export"

#OVERRIDES = PATH + "pypsa-eur-sec/data/override_component_attrs"

PREFIX = "/mnt/c/Users/scl38887/Documents/git/pypsa-nz/"
INPUT = PREFIX + f"workflow/subworkflows/pypsa-earth-sec/results/{RUN}/{CATEGORY}/{SCENARIO}.nc"

In [ ]:
config_aldehyde = PREFIX + "config/config.yaml"

In [ ]:
threshold_dispatch = 1e6 # 4 TWh
h2export = 200
opts = "Co2L"
SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_{opts}_{H}H_2030_{DISCOUNT_RATE}_AB_{h2export}export"


In [ ]:
RUNS = {"0": "nz_erafixtwo",
        "20": "nz_erafixtwo",
        "200": "nz_erafixtwo",
       }

scenarios = ["0","20", "200"]

# PREFIX = "/mnt/c/Users/scl38887/Documents/git/aldehyde/"
# OUTPUT = PREFIX + "results/gh/"
# config_aldehyde = PREFIX + "config/config.yaml"

### Data preparation

In [ ]:
def load_config(config_aldehyde):
    with open(config_aldehyde, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

In [ ]:
config = load_config(config_aldehyde)

### Plot dispatch

In [ ]:
def get_dispatch(threshold_dispatch, ghrules, h2export, opts, show_demand=True):
    """Get dispatch
    """

    dispatch_all = pd.DataFrame()

    for sc in scenarios:

        SCENARIO = f"elec_s_{CLUSTERS}_ec_lc{LL}_{opts}_{H}H_2030_{DISCOUNT_RATE}_AB_{sc}export"
        INPUT = PREFIX + f"results/{RUNS[sc]}/{CATEGORY}/{SCENARIO}.nc"

        n = pypsa.Network(INPUT)

        dispatch = n.statistics.dispatch(bus_carrier="AC")[n.statistics.dispatch(bus_carrier="AC") > 0]
        threshold_dispatch_sum = dispatch[dispatch < threshold_dispatch].sum()
        dispatch = dispatch[dispatch > threshold_dispatch]
        dispatch = pd.DataFrame(dispatch.droplevel(0)).T
        dispatch[f"(Dispatch < {threshold_dispatch/1e6} TWh thres.)"] = threshold_dispatch_sum

        dispatch.index = [sc]

        dispatch_all = pd.concat([dispatch_all, dispatch], axis=0) #, ignore_index=True)

    if show_demand == "bar":
        dispatch_demand = n.statistics.dispatch(bus_carrier="AC")[n.statistics.dispatch(bus_carrier="AC") < 0]
        ely_electricity = dispatch_demand.droplevel(0).loc["H2 Electrolysis"].sum() * (-1)
        total_demand = dispatch_demand.sum() * (-1)
        other_demand = total_demand - ely_electricity
        dispatch_all.loc["Demand", "H2 Electrolysis electricity"] = ely_electricity
        dispatch_all.loc["Demand", "Domestic electricity"] = other_demand
        
    elif show_demand == "line":
        dispatch_demand = n.statistics.dispatch(bus_carrier="AC")[n.statistics.dispatch(bus_carrier="AC") < 0]
        ely_electricity = dispatch_demand.droplevel(0).loc["H2 Electrolysis"].sum() * (-1)
        total_demand = dispatch_demand.sum() * (-1)
        other_demand = total_demand - ely_electricity
        dispatch_all.loc[:, "H2 Electrolysis electricity"] = ely_electricity
        dispatch_all.loc[:, "Domestic electricity"] = other_demand
        
    
    return dispatch_all

In [ ]:
dispatch = get_dispatch(threshold_dispatch, scenarios, h2export, opts, show_demand="bar")

In [ ]:
dispatch

In [ ]:
#df = prepare_data(ghrules, h2export, opts)

fig, ax = plt.subplots(figsize=(6, 4))

colors = config["plot"]["tech_colors"]
# Sort columns by highest value in first row
dispatch_sorted = dispatch[dispatch.iloc[3].sort_values(ascending=False).index]
dispatch_twh = dispatch_sorted / 1e6 # in TWh

# This line in case of electricty line not bar
# dispatch_twh = dispatch_twh.iloc[:, ~dispatch_twh.columns.isin(["H2 Electrolysis electricity", "Domestic electricity"])]
# dispatch[["H2 Electrolysis electricity", "Domestic electricity"]].div(1e6).plot.line(ax=ax, alpha=1, color=colors)

dispatch_twh.plot.bar(ax=ax, stacked=True, alpha=0.8, color=colors)

#plt.xlabel("H2 rule")
plt.ylabel("Dispatch/Demand in TWh")
plt.title(f"{h2export} TWh export, {opts} CO2")
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=0)
plt.ylim(0, 250)

plt.grid(axis='y', alpha=0.5) 

fig.savefig(OUTPUT+f"dispatch_{h2export}_{opts}.pdf", bbox_inches="tight")